In [20]:
import skvideo
print(skvideo.__file__)
skvideo._FFMPEG_SUPPORTED_ENCODERS = {}
print(skvideo._FFMPEG_PATH)
#skvideo.setFFmpegPath("/usr/bin/ffprobe")
skvideo.scan_ffmpeg()
print(skvideo._FFMPEG_SUPPORTED_ENCODERS)


/home/isaac/miniconda2/lib/python2.7/site-packages/skvideo/__init__.pyc
/usr/bin/ffprobe
['., A64', '.264', '.265', '.302', '.3g2', '.3gp', '.722', '.a64', '.aa3', '.aac', '.ac3', '.adts', '.adx', '.afc', '.aif', '.aifc', '.aiff', '.al', '.amr', '.apng', '.asf', '.ass', '.ast', '.au', '.avc', '.avi', '.bit', '.bmp', '.caf', '.cavs', '.chk', '.cif', '.daud', '.dif', '.dnxhd', '.dpx', '.drc', '.dts', '.dv', '.dvd', '.eac3', '.f4v', '.ffm', '.ffmeta', '.flac', '.flm', '.flv', '.g722', '.g723_1', '.gif', '.gxf', '.h261', '.h263', '.h264', '.h265', '.h26l', '.hevc', '.ico', '.im1', '.im24', '.im8', '.ircam', '.isma', '.ismv', '.ivf', '.j2c', '.j2k', '.jls', '.jp2', '.jpeg', '.jpg', '.js', '.jss', '.latm', '.lbc', '.ljpg', '.loas', '.lrc', '.m1v', '.m2a', '.m2t', '.m2ts', '.m2v', '.m3u8', '.m4a', '.m4v', '.mj2', '.mjpeg', '.mjpg', '.mk3d', '.mka', '.mks', '.mkv', '.mlp', '.mmf', '.mov', '.mp2', '.mp3', '.mp4', '.mpa', '.mpeg', '.mpg', '.mpo', '.mts', '.mxf', '.nut', '.oga', '.ogg', '.ogv', '

In [1]:
import skvideo
import skvideo.io
import skvideo.datasets

import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib import gridspec
import sklearn.linear_model
from sklearn.feature_extraction import image

print(skvideo._FFMPEG_SUPPORTED_ENCODERS)

np.random.seed(0)

# use greedy K-SVD algorithm with OMP
def code_step(X, D):
  model = sklearn.linear_model.OrthogonalMatchingPursuit(
          n_nonzero_coefs=5, fit_intercept=False, normalize=False
  )
  #C = sklearn.
  model.fit(D.T, X.T)
  return model.coef_

def dict_step(X, C, D):
  unused_indices = []
  for k in xrange(D.shape[0]):
    usedidx = np.abs(C[:, k])>0

    if np.sum(usedidx) <= 1:
      print "Skipping filter #%d" % (k,)
      unused_indices.append(k)
      continue

    selectNotK = np.arange(D.shape[0]) != k
    used_coef = C[usedidx, :][:, selectNotK]

    E_kR = X[usedidx, :].T - np.dot(used_coef, D[selectNotK, :]).T

    U, S, V = scipy.sparse.linalg.svds(E_kR, k=1)

    # choose sign based on largest dot product
    choicepos = np.dot(D[k,:], U[:, 0])
    choiceneg = np.dot(D[k,:], -U[:, 0])

    if choicepos > choiceneg:
      D[k, :] = U[:, 0]
      C[usedidx, k] = S[0] * V[0, :]
    else:
      D[k, :] = -U[:, 0]
      C[usedidx, k] = -S[0] * V[0, :]


  # re-randomize filters that were not used
  for i in unused_indices:
    D[i, :] = np.random.normal(size=D.shape[1])
    D[i, :] /= np.sqrt(np.dot(D[i,:], D[i,:]))

  return D

def plot_weights(basis):
    n_filters, n_channels, height, width = basis.shape
    ncols = 10
    nrows = 10
    fig = plt.figure()
    gs = gridspec.GridSpec(nrows, ncols)
    rown = 0
    coln = 0
    for filter in xrange(n_filters):
            ax = fig.add_subplot(gs[rown, coln])
            mi = np.min(basis[filter, 0, :, :])
            ma = np.max(basis[filter, 0, :, :])
            ma = np.max((np.abs(mi), np.abs(ma)))
            mi = -ma
            ax.imshow(basis[filter, 0, :, :], vmin=mi, vmax=ma, cmap='Greys_r', interpolation='none')
            ax.xaxis.set_major_locator(plt.NullLocator())
            ax.yaxis.set_major_locator(plt.NullLocator())
            coln += 1
            if coln >= ncols:
                coln = 0
                rown += 1
    gs.tight_layout(fig, pad=0, h_pad=0, w_pad=0)
    fig.canvas.draw()
    buf, sz = fig.canvas.print_to_buffer()
    data = np.fromstring(buf, dtype=np.uint8).reshape(sz[1], sz[0], -1)[:, :, :3]
    plt.close()
    return data

# a 5 fps video encoded using x264
writer = skvideo.io.FFmpegWriter("sparsity.mp4", 
  inputdict={
    "-r": "10"
  },
  outputdict={
  '-vcodec': 'libx264', '-b': '30000000'
})

# open the first frame of bigbuckbunny
filename = skvideo.datasets.bigbuckbunny()
vidframe = skvideo.io.vread(filename, outputdict={"-pix_fmt": "gray"})[0, :, :, 0]

# initialize D
D = np.random.normal(size=(100, 7*7))
for i in range(D.shape[0]):
  D[i, :] /= np.sqrt(np.dot(D[i,:], D[i,:]))


X = image.extract_patches_2d(vidframe, (7, 7))
X = X.reshape(X.shape[0], -1).astype(np.float)

# sumsample about 10000 patches
X = X[np.random.permutation(X.shape[0])[:10000]]

for i in range(200):
  print "Iteration %d / %d" % (i, 200)
  C = code_step(X, D)
  D = dict_step(X, C, D)
  frame = plot_weights(D.reshape(100, 1, 7, 7))
  writer.writeFrame(frame)
writer.close()

['., A64', '.264', '.265', '.302', '.3g2', '.3gp', '.722', '.a64', '.aa3', '.aac', '.ac3', '.adts', '.adx', '.afc', '.aif', '.aifc', '.aiff', '.al', '.amr', '.apng', '.asf', '.ass', '.ast', '.au', '.avc', '.avi', '.bit', '.bmp', '.caf', '.cavs', '.chk', '.cif', '.daud', '.dif', '.dnxhd', '.dpx', '.drc', '.dts', '.dv', '.dvd', '.eac3', '.f4v', '.ffm', '.ffmeta', '.flac', '.flm', '.flv', '.g722', '.g723_1', '.gif', '.gxf', '.h261', '.h263', '.h264', '.h265', '.h26l', '.hevc', '.ico', '.im1', '.im24', '.im8', '.ircam', '.isma', '.ismv', '.ivf', '.j2c', '.j2k', '.jls', '.jp2', '.jpeg', '.jpg', '.js', '.jss', '.latm', '.lbc', '.ljpg', '.loas', '.lrc', '.m1v', '.m2a', '.m2t', '.m2ts', '.m2v', '.m3u8', '.m4a', '.m4v', '.mj2', '.mjpeg', '.mjpg', '.mk3d', '.mka', '.mks', '.mkv', '.mlp', '.mmf', '.mov', '.mp2', '.mp3', '.mp4', '.mpa', '.mpeg', '.mpg', '.mpo', '.mts', '.mxf', '.nut', '.oga', '.ogg', '.ogv', '.oma', '.omg', '.opus', '.pam', '.pbm', '.pcx', '.pgm', '.pgmyuv', '.pix', '.png', '.ppm'